In [34]:
import pandas as pd
import os
import sys
from pathlib import Path
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

project_dir = Path('__main__').resolve().parents[1]

In [2]:
df_test = pd.read_csv(os.path.join(project_dir, 'data', 'raw', 'fraudTest.csv'))
df_train = pd.read_csv(os.path.join(project_dir, 'data', 'raw', 'fraudTrain.csv'))

In [3]:
df_train['ano_mes_data'] = df_train['trans_date_trans_time'].astype('datetime64[ns]')
df_train['ano_mes'] = df_train['trans_date_trans_time'].str[0:7]

In [4]:
df_test['ano_mes_data'] = df_test['trans_date_trans_time'].astype('datetime64[ns]')
df_test['ano_mes'] = df_test['trans_date_trans_time'].str[0:7]

In [5]:
cc_nums_train = df_train.groupby('cc_num').ano_mes.nunique().sort_values(ascending=False)[0:200].index

In [6]:
cc_nums_test = df_test.groupby('cc_num').ano_mes.nunique().sort_values(ascending=False)[0:100].index

In [7]:
df_train = df_train[((df_train.cc_num.isin(cc_nums_train))| (df_train.cc_num.isin(cc_nums_test))) &
                    (df_train.ano_mes_data < pd.to_datetime('2020-06-01 00:00:00'))
                    ]

In [8]:
df_test = df_test[(df_test.cc_num.isin(cc_nums_test)) &
                    (df_test.ano_mes_data >= pd.to_datetime('2020-06-01 00:00:00')) 
                ]

In [9]:
df = pd.concat([df_train, df_test], ignore_index=True)

In [10]:
del df_train, df_test

In [11]:
print(df.shape, df.is_fraud.mean())
df.head()

(345938, 25) 0.004934410212234563


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,ano_mes_data,ano_mes
0,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,2019-01-01 00:01:16,2019-01
1,14,2019-01-01 00:09:03,3514865930894695,fraud_Beier-Hyatt,shopping_pos,7.77,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,...,899,Naval architect,1967-08-30,8a6293af5ed278dea14448ded2685fea,1325376543,32.863258,-106.520205,0,2019-01-01 00:09:03,2019-01
2,27,2019-01-01 00:21:32,4334230547694630,fraud_Bruen-Yost,misc_pos,6.85,Scott,Martin,M,7483 Navarro Flats,...,471,"Education officer, museum",1967-08-02,f3c43d336e92a44fc2fb67058d5949e3,1325377292,43.753735,-111.454923,0,2019-01-01 00:21:32,2019-01
3,29,2019-01-01 00:22:18,4260059589824237,"fraud_Nitzsche, Kessler and Wolff",shopping_pos,4.02,Aaron,Rogers,M,969 Huerta Path Apt. 270,...,4005,Network engineer,1945-03-15,20490f3f0966ce74b4aaba8dc2c4ed52,1325377338,42.475590,-101.265846,0,2019-01-01 00:22:18,2019-01
4,36,2019-01-01 00:26:22,3598215285024754,fraud_Heidenreich PLC,grocery_pos,207.36,Ashley,Lopez,F,9333 Valentine Point,...,34496,"Librarian, public",1970-10-21,f246ac27c09429ff1e385d4683c79c4d,1325377582,40.202538,-73.859212,0,2019-01-01 00:26:22,2019-01


# Window Features

In [12]:
def p99(x):
    return x.quantile(0.99)

def p90(x):
    return x.quantile(0.9)

def p75(x):
    return x.quantile(0.75)

def p01(x):
    return x.quantile(0.01)

def p10(x):
    return x.quantile(0.1)

def p25(x):
    return x.quantile(0.25)

In [13]:
df_top_mcc = df.groupby(['cc_num', 'ano_mes','category']).agg({'amt' : ['count', 'sum', 'max', 'min', p99, p90, p75, 'median', p25, p10, p01]})

In [14]:
df_top_mcc = df_top_mcc.reset_index()

In [16]:
df_top_mcc.columns = ['cc_num', 'ano_mes', 'category', 'count_mcc', 'sum_mcc', 'max_mcc', 'min_mcc', 'p99_mcc', 'p90_mcc', 'p75_mcc', 'median_mcc', 'p25_mcc', 'p10_mcc', 'p01_mcc']

In [17]:
df_top_mcc.head()

,cc_num,ano_mes,category,count_mcc,sum_mcc,max_mcc,min_mcc,p99_mcc,p90_mcc,p75_mcc,median_mcc,p25_mcc,p10_mcc,p01_mcc
0,60416207185,2019-01,entertainment,3,151.92,81.48,12.83,81.0026,76.706,69.545,57.610,35.2200,21.786,13.7256
1,60416207185,2019-01,food_dining,4,92.30,73.85,3.06,71.8874,54.224,24.785,7.695,5.9850,4.230,3.1770
2,60416207185,2019-01,gas_transport,6,356.31,82.08,42.70,81.2275,73.555,64.850,58.625,50.1725,45.975,43.0275
3,60416207185,2019-01,grocery_pos,5,477.90,121.09,72.81,120.4544,114.734,105.200,102.470,76.3300,74.218,72.9508
4,60416207185,2019-01,health_fitness,3,227.59,156.15,17.55,154.1048,135.698,105.020,53.890,35.7200,24.818,18.2768


In [47]:
df_top3_mcc = (df.groupby(['cc_num', 'ano_mes', 'category'])
               .size().reset_index(name='count')
               .sort_values(['cc_num', 'ano_mes', 'count'], ascending=[True, False, False])
               .groupby(['cc_num', 'ano_mes']).head(3).reset_index(drop=True))

In [48]:
df_top3_mcc['row_number'] = df_top3_mcc.groupby(['cc_num', 'ano_mes']).cumcount() + 1

In [49]:
df_top3_mcc.columns = ['cc_num', 'ano_mes', 'category', 'count', 'row_number']

In [36]:
df_cpf = df.groupby(['cc_num', 'ano_mes']).agg({'amt' : ['count', 'sum', 'max', 'min', p99, p90, p75, 'median', p25, p10, p01]})

In [37]:
df_cpf = df_cpf.reset_index()

In [40]:
df_cpf.columns = ['cc_num', 'ano_mes', 'count_cpf', 'sum_cpf', 'max_cpf', 'min_cpf', 'p99_cpf', 'p90_cpf', 'p75_cpf', 'median_cpf', 'p25_cpf', 'p10_cpf', 'p01_cpf']

In [41]:
df_cpf.head()

,cc_num,ano_mes,count_cpf,sum_cpf,max_cpf,min_cpf,p99_cpf,p90_cpf,p75_cpf,median_cpf,p25_cpf,p10_cpf,p01_cpf
0,60416207185,2019-01,56,2410.54,204.15,1.84,202.2910,104.935,64.4900,21.685,6.9375,3.470,1.8730
1,60416207185,2019-02,59,2861.48,224.75,1.24,210.8764,118.436,74.6100,29.220,10.0500,4.592,1.4198
2,60416207185,2019-03,86,6672.74,852.81,1.07,833.4725,133.555,82.4025,48.865,10.1000,4.255,1.5375
3,60416207185,2019-04,102,6430.91,698.09,1.16,470.5620,117.461,73.3950,43.280,8.9100,4.517,1.4201
4,60416207185,2019-05,81,3543.45,225.48,1.22,223.9360,112.830,63.2600,34.230,6.1000,3.930,1.3080


In [65]:
df_abt_train = df[df.ano_mes_data < pd.to_datetime('2020-06-01')].groupby(['cc_num', 'ano_mes']).agg({'is_fraud' : 'max'}).reset_index()

In [66]:
df_abt_test = df[df.ano_mes_data >= pd.to_datetime('2020-06-01')].groupby(['cc_num', 'ano_mes']).agg({'is_fraud' : 'max'}).reset_index()

In [67]:
df_abt_train.agg({'is_fraud' : ['count', 'sum', 'mean']}), df_abt_test.agg({'is_fraud' : ['count', 'sum', 'mean']})

(          is_fraud
 count  3468.000000
 sum     147.000000
 mean      0.042388,
          is_fraud
 count  700.000000
 sum     23.000000
 mean     0.032857)

# Merge Feature Data

In [51]:
df_cpf_merged = df_cpf.merge(df_top3_mcc[df_top3_mcc.row_number == 1][['cc_num', 'category', 'ano_mes']], how='left', on=['cc_num', 'ano_mes'])

In [53]:
df_cpf_merged = df_cpf_merged.merge(df_top3_mcc[df_top3_mcc.row_number == 2][['cc_num', 'category', 'ano_mes']], how='left', on=['cc_num', 'ano_mes'], suffixes=('', 'top2'))

In [54]:
df_cpf_merged = df_cpf_merged.merge(df_top3_mcc[df_top3_mcc.row_number == 3][['cc_num', 'category', 'ano_mes']], how='left', on=['cc_num', 'ano_mes'], suffixes=('top1', 'top3'))

In [55]:
df_cpf_merged = df_cpf_merged.rename(columns={'categorytop1' : 'category'}).merge(df_top_mcc, how='left', on=['cc_num', 'ano_mes', 'category']).rename(columns={'category' : 'categorytop1'})

In [56]:
df_cpf_merged = df_cpf_merged.rename(columns={'categorytop2' : 'category'}).merge(df_top_mcc, how='left', on=['cc_num', 'ano_mes', 'category'], suffixes=('', '_top2')).rename(columns={'category' : 'categorytop2'})

In [57]:
df_cpf_merged = df_cpf_merged.rename(columns={'categorytop3' : 'category'}).merge(df_top_mcc, how='left', on=['cc_num', 'ano_mes', 'category'], suffixes=('_top1', '_top3')).rename(columns={'category' : 'categorytop3'})

In [59]:
df_cpf_merged.head()

,cc_num,ano_mes,count_cpf,sum_cpf,max_cpf,min_cpf,p99_cpf,p90_cpf,p75_cpf,median_cpf,p25_cpf,p10_cpf,p01_cpf,categorytop1,categorytop2,categorytop3,count_mcc_top1,sum_mcc_top1,max_mcc_top1,min_mcc_top1,p99_mcc_top1,p90_mcc_top1,p75_mcc_top1,median_mcc_top1,p25_mcc_top1,p10_mcc_top1,p01_mcc_top1,count_mcc_top2,sum_mcc_top2,max_mcc_top2,min_mcc_top2,p99_mcc_top2,p90_mcc_top2,p75_mcc_top2,median_mcc_top2,p25_mcc_top2,p10_mcc_top2,p01_mcc_top2,count_mcc_top3,sum_mcc_top3,max_mcc_top3,min_mcc_top3,p99_mcc_top3,p90_mcc_top3,p75_mcc_top3,median_mcc_top3,p25_mcc_top3,p10_mcc_top3,p01_mcc_top3
0,60416207185,2019-01,56,2410.54,204.15,1.84,202.2910,104.935,64.4900,21.685,6.9375,3.470,1.8730,personal_care,home,gas_transport,10,266.23,104.67,3.17,98.8299,46.269,33.490,20.715,5.0250,4.421,3.2951,7,304.63,117.11,4.84,113.8406,84.416,57.5450,27.180,20.205,14.050,5.7610,6,356.31,82.08,42.70,81.2275,73.555,64.850,58.625,50.1725,45.975,43.0275
1,60416207185,2019-02,59,2861.48,224.75,1.24,210.8764,118.436,74.6100,29.220,10.0500,4.592,1.4198,entertainment,gas_transport,home,7,258.92,90.60,7.27,89.4318,78.918,60.715,15.320,12.1500,9.268,7.4698,6,388.24,84.42,32.97,84.1285,81.505,76.1525,67.885,59.100,44.730,34.1460,6,514.19,200.83,12.39,198.6955,179.485,140.365,65.145,20.2175,12.465,12.3975
2,60416207185,2019-03,86,6672.74,852.81,1.07,833.4725,133.555,82.4025,48.865,10.1000,4.255,1.5375,gas_transport,grocery_pos,home,20,1155.00,103.50,11.74,100.6747,83.671,69.290,54.910,48.3325,33.798,12.0117,9,899.36,154.46,77.54,150.8760,118.620,106.8300,98.230,83.310,77.828,77.5688,8,598.53,261.79,1.07,251.0107,153.997,89.980,53.735,22.8300,6.537,1.6167
3,60416207185,2019-04,102,6430.91,698.09,1.16,470.5620,117.461,73.3950,43.280,8.9100,4.517,1.4201,gas_transport,shopping_pos,home,19,1079.29,76.55,39.05,76.1540,71.294,61.735,57.110,49.9350,44.738,39.3740,15,956.42,472.17,3.12,449.6580,225.422,9.6250,6.430,5.320,4.080,3.2460,9,540.75,163.17,4.21,158.5996,117.466,104.690,48.060,13.5300,4.410,4.2300
4,60416207185,2019-05,81,3543.45,225.48,1.22,223.9360,112.830,63.2600,34.230,6.1000,3.930,1.3080,gas_transport,shopping_pos,entertainment,15,797.17,73.26,26.35,73.2530,70.226,64.295,53.940,46.6350,33.138,27.1984,11,174.85,116.73,1.22,106.0170,9.600,8.0200,6.460,4.870,2.440,1.3420,7,233.78,129.06,3.35,123.6114,74.574,33.080,14.180,10.5150,6.476,3.6626


In [68]:
df_cpf_merged.to_parquet(os.path.join(project_dir, 'data', 'interim', 'features.parquet.gzip'), compression='gzip', index=False)

In [69]:
df_abt_train.to_parquet(os.path.join(project_dir, 'data', 'interim', 'abt_train.parquet.gzip'), compression='gzip', index=False)

In [70]:
df_abt_test.to_parquet(os.path.join(project_dir, 'data', 'interim', 'abt_test.parquet.gzip'), compression='gzip', index=False)

# Trasnform

In [ ]:
#Seletor
#Ordinal Encoder
#MinMax